# IUPred - implementation


In this project I was implementing the IUPred predictor in a clean Pythonic way... as far as I could.

In [140]:
import pandas as pd
import sys
from collections import defaultdict
import statistics

In [5]:
def save_output(
    data : pd.DataFrame, 
    iupred_type: str, 
    outfile : str = None) -> None:
    '''There is a fixed header printed on all the output + the tsv of the output.
    '''
    header = ('# IUPred2A: context-dependent prediction of protein disorder as a function of redox state and protein binding\n'
    "# Balint Meszaros, Gabor Erdos, Zsuzsanna Dosztanyi\n"
    "# Nucleic Acids Research 2018;46(W1):W329-W337.\n"
    "#\n"
    f"# Prediction type: {iupred_type}\n"
    "# Prediction output\n")
    
    # Output is directed either to a file or to the standard output:
    if outfile:
        with open(outfile, 'w') as f:
                f.write(header)
                data.to_csv(f, sep='\t', index=False)
    else:
        print(header + df.to_csv(sep='\t', index=False))
    
    return None

In [21]:
def read_fasta(fasta_file: str) -> dict:
    '''This function returns with a dictionary of the sequences in a fasta file. the keys are the header row'''
    sequences = {}

    with open(fasta_file) as f:
        for line in f:
            # Found a header:
            if line.startswith(">"):
                key = line.strip()
                sequences[key] = ''

            # Add rows to a given header:
            else:
                sequences[key] += line.strip()
    
    return sequences


seqence = read_fasta('single.fasta')
seq = list(seqence.values())[0]
seq

'MNRHLWKSQLCEMVQPSGGPAADQDVLGEESPLGKPAMLHLPSEQGAPETLQRCLEENQELRDAIRQSNQILRERCEELLHFQASQREEKEFLMCKFQEARKLVERLGLEKLDLKRQKEQALREVEHLKRCQQQMAEDKASVKAQVTSLLGELQESQSRLEAATKECQALEGRARAASEQARQLESEREALQQQHSVQVDQLRMQGQSVEAALRMERQAASEEKRKLAQLQVAYHQLFQEYDNHIKSSVVGSERKRGMQLEDLKQQLQQAEEALVAKQEVIDKLKEEAEQHKIVMETVPVLKAQADIYKADFQAERQAREKLAEKKELLQEQLEQLQREYSKLKASCQESARIEDMRKRHVEVSQAPLPPAPAYLSSPLALPSQRRSPPEEPPDFCCPKCQYQAPDMDTLQIHVMECIE'

In [7]:
# from collections import defaultdict

def aa_freq(_seq):
    _freq = defaultdict(int)
    for _aa in _seq:
        _freq[_aa] += 1
    
    for _aa, _ins in _freq.items():
        _freq[_aa] = _ins / len(_seq)
    return _freq

def read_matrix(matrix_file):
    _mtx = defaultdict(defaultdict)
    with open(matrix_file, "r") as _fhm:
        for _line in _fhm:
            (_aa1, _aa2, _val) = _line.split()
            _mtx[_aa1][_aa2] = float(_val)

    return _mtx


# read_matrix('/Users/dsuveges/Downloads/iupred2a/data/iupred2_long_energy_matrix')
def read_histo(histo_file):
    hist = []
    vals = []

    with open(histo_file, "r") as fnh:
        for _line in fnh:
            if _line.startswith("#"):
                continue
                
            vals.append(float(_line.split()[1]))
            hist.append(float(_line.split()[-1]))
            
    h_max = max(vals)
    h_min = min(vals)
    h_step = (h_max - h_min) / (len(hist))
    return hist, h_min, h_max, h_step

(hist, h_min, h_max, h_step) = read_histo('/Users/dsuveges/Downloads/iupred2a/data/long_histogram')

In [186]:
import pandas as pd
import sys
from collections import defaultdict
import statistics



class iupred2a(object):
    
    PATH = '/Users/dsuveges/repositories/iupred2a/data'

    def __init__(self, method='long'):
        
        # Validating iupred method:
        if iupred_type not in ['long', 'short', 'glob']:
            raise TypeError(f'Unknown prediction type: {iupred_type}. Supported types: "long", "short", "glob"')
        
        # Store iupred method:
        self.method = method
        
        # Store common parameter:
        self.lc = 1
        
        # Set method specific parameters:
        if method == "short":
            self.uc = 25
            self.wc = 10

        elif method == 'glob':
            self.uc = 100
            self.wc = 15

        elif method == 'long':
            self.uc = 100
            self.wc = 10
        
        # Set method specific histograms and matrices:
        if method == "short":
            matrix_file = f"{self.PATH}/iupred2_short_energy_matrix"
            histogram_file = f"{self.PATH}/short_histogram"

        elif method == "long" or iupred_type == "glob":
            matrix_file = f"{self.PATH}/iupred2_long_energy_matrix"
            histogram_file = f"{self.PATH}/long_histogram"   
        
        # Load matrix and histogram:
        self.matrix = self.read_matrix(matrix_file)
        (self.histogram, self.histogram_min, self.histogram_max, self.histogram_step) = read_histo(histogram_file)


    @staticmethod
    def read_matrix(matrix_file):
        _mtx = defaultdict(defaultdict)
        with open(matrix_file, "r") as _fhm:
            for _line in _fhm:
                (_aa1, _aa2, _val) = _line.split()
                _mtx[_aa1][_aa2] = float(_val)

        return _mtx


    @staticmethod
    def read_histo(histo_file):
        hist = []
        vals = []

        with open(histo_file, "r") as fnh:
            for _line in fnh:
                if _line.startswith("#"):
                    continue

                vals.append(float(_line.split()[1]))
                hist.append(float(_line.split()[-1]))

        h_max = max(vals)
        h_min = min(vals)
        h_step = (h_max - h_min) / (len(hist))
        return hist, h_min, h_max, h_step       
    
    
    @staticmethod
    def aa_freq(_seq):
        _freq = defaultdict(int)
        for _aa in _seq:
            _freq[_aa] += 1

        for _aa, _ins in _freq.items():
            _freq[_aa] = _ins / len(_seq)
        return _freq
    
    
    @staticmethod
    def smooth(energy_list, window):
        
        weighted_energy_score = [0] * len(energy_list)
        for idx in range(len(energy_list)):
            weighted_energy_score[idx] = statistics.mean(energy_list[max(0, idx - window):min(len(energy_list),idx + window + 1)])
        return weighted_energy_score
    

    def calculate_iupred_score(self):
        iupred_score = [0] * len(self.weighted_energy_score)
        for idx, val in enumerate(self.weighted_energy_score):
            if val <= self.histogram_min + 2 * self.histogram_step:
                iupred_score[idx] = 1
            elif val >= self.histogram_max - 2 * self.histogram_step:
                iupred_score[idx] = 0
            else:
                iupred_score[idx] = self.histogram[int((weighted_energy_score[idx] - self.histogram_min) * (1 / self.histogram_step))]

        return iupred_score
    

    def predict(self, seq):
        self.unweighted_energy_score = self.calculate_unweighted_energy_score(seq)
        self.weighted_energy_score = self.calculate_weighted_energy_score(seq)
        
        # Calculate IUPred scores:
        if self.method == 'long' or self.method == 'short':
            return self.calculate_iupred_score()
        
        # Calcualte Globular domain:
        if self.method == 'glob':
            return self.calculate_globular()
    

    def calculate_weighted_energy_score(self, seq):
        weighted_energy_score = [0] * len(seq)
        if self.method == 'short':
            for idx in range(len(seq)):
                for idx2 in range(idx - self.wc, idx + self.wc + 1):
                    if idx2 < 0 or idx2 >= len(seq):
                        weighted_energy_score[idx] += -1.26
                    else:
                        weighted_energy_score[idx] += self.unweighted_energy_score[idx2]
                weighted_energy_score[idx] /= len(range(idx - self.wc, idx + self.wc + 1))
        else:
            weighted_energy_score = self.smooth(self.unweighted_energy_score, self.wc)

        return weighted_energy_score
    

    def calculate_unweighted_energy_score(self, seq):
        unweighted_energy_score = [0] * len(seq)
        for idx in range(len(seq)):
            freq_dct = self.aa_freq(seq[max(0, idx - self.uc):max(0, idx - self.lc)] + seq[idx + self.lc + 1:idx + self.uc + 1])
            for aa, freq in freq_dct.items():
                unweighted_energy_score[idx] += mtx[seq[idx]][aa] * freq
        return unweighted_energy_score
        

    def calculate_globular(self, seq):
        # Glob mode
        glob_text = ""
        gr = []
        in_gr = False
        
        beg, end = 0, 0
        for idx, val in enumerate(self.weighted_energy_score):
            if in_gr and val <= 0.3:
                gr.append({0: beg, 1: end})
                in_gr = False
            elif in_gr:
                end += 1
            if val > 0.3 and not in_gr:
                beg = idx
                end = idx
                in_gr = True
        if in_gr:
            gr.append({0: beg, 1: end})
        mgr = []
        k = 0
        kk = k + 1
        beg = gr[0][0]
        end = gr[0][1]
        nr = len(gr)
        while k < nr:
            if kk < nr and gr[kk][0] - end < 45:
                beg = gr[k][0]
                end = gr[kk][1]
                kk += 1
            elif end - beg + 1 < 35:
                k += 1
                if k < nr:
                    beg = gr[k][0]
                    end = gr[k][1]
            else:
                mgr.append({0: beg, 1: end})
                k = kk
                kk += 1
                if k < nr:
                    beg = gr[k][0]
                    end = gr[k][1]
        seq = seq.lower()
        nr = 0
        res = ""
        for i in mgr:
            res += seq[nr:i[0]] + seq[i[0]:i[1] + 1].upper()
            nr = i[1] + 1
        res += seq[nr:]
        res = " ".join([res[i:i + 10] for i in range(0, len(res), 10)])
        glob_text += "Number of globular domains: {}\n".format(len(mgr))
        for n, i in enumerate(mgr):
            glob_text += "          globular domain   {}.\t{}-{}\n".format(n + 1, i[0] + 1, i[1] + 1)
        glob_text += "\n".join(textwrap.wrap(res, 70))
        return iupred_score, glob_text


In [182]:
seq

'MNRHLWKSQLCEMVQPSGGPAADQDVLGEESPLGKPAMLHLPSEQGAPETLQRCLEENQELRDAIRQSNQILRERCEELLHFQASQREEKEFLMCKFQEARKLVERLGLEKLDLKRQKEQALREVEHLKRCQQQMAEDKASVKAQVTSLLGELQESQSRLEAATKECQALEGRARAASEQARQLESEREALQQQHSVQVDQLRMQGQSVEAALRMERQAASEEKRKLAQLQVAYHQLFQEYDNHIKSSVVGSERKRGMQLEDLKQQLQQAEEALVAKQEVIDKLKEEAEQHKIVMETVPVLKAQADIYKADFQAERQAREKLAEKKELLQEQLEQLQREYSKLKASCQESARIEDMRKRHVEVSQAPLPPAPAYLSSPLALPSQRRSPPEEPPDFCCPKCQYQAPDMDTLQIHVMECIE'

In [188]:
ip = iupred2a('short')
ip.predict(seq)

defaultdict(<class 'collections.defaultdict'>, {'F': defaultdict(None, {'F': 5.24746144, 'G': -0.17828237, 'L': 5.54998578, 'Y': 4.64434088, 'K': 0.46606451, 'D': -0.24482101, 'W': 3.11777737, 'M': 3.87839804, 'N': 0.05614576, 'R': 0.81115817, 'S': 1.02681044, 'I': 3.84372701, 'T': 0.14088095, 'V': 5.20931719, 'E': -1.29915607, 'P': -1.31624529, 'Q': -0.87607973, 'C': 3.22136449, 'H': 2.44199924, 'A': 2.15109881}), 'G': defaultdict(None, {'F': -0.17828237, 'G': 0.14223602, 'L': -0.13036212, 'Y': 1.46825645, 'K': -0.13027794, 'D': -0.80605606, 'W': -1.08948886, 'M': 0.06215472, 'N': 0.17280222, 'R': -0.90623557, 'S': -0.58072784, 'I': -0.0707368, 'T': -0.48489851, 'V': -0.16637907, 'E': -1.13025024, 'P': -1.12692286, 'Q': -1.21558991, 'C': 2.21534292, 'H': -0.65396747, 'A': 0.41919169}), 'L': defaultdict(None, {'F': 5.54998578, 'G': -0.13036212, 'L': 5.61160378, 'Y': 3.50131333, 'K': -0.47012503, 'D': -0.1618335, 'W': 4.70049024, 'M': 1.80808171, 'N': -0.3066697, 'R': 0.20153959, 'S': -

[0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 0.0033,
 

In [177]:
iupred = iu.predict(seq)
type(iupred)

[0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352, 0.045352,

NoneType

In [71]:
idx = 100
seq_chunk = seq[max(0, idx - uc):max(0, idx - lc)] + seq[idx + lc + 1:idx + uc + 1]
print(len(seq_chunk))
frec_dict = aa_freq(seq[max(0, idx - uc):max(0, idx - lc)] + seq[idx + lc + 1:idx + uc + 1])

48


In [115]:
matrix = pd.read_csv('/Users/dsuveges/repositories/iupred2a/data/iupred2_long_energy_matrix', sep=' ', names=['ref','neighbour','energy'])

# unweighted_energy_score[idx] = (
#     matrix.loc[matrix.ref==seq[idx]]
#     .apply(lambda x: frec_dict[x['lookup']] * x['energy'] if x['lookup'] in frec_dict else 0, axis=1)
#     .sum()
# )

In [108]:
unweighted_energy_score[idx] = 0
for aa, freq in frec_dict.items():
    unweighted_energy_score[idx] += mtx[seq[idx]][aa] * freq
    
unweighted_energy_score[idx]

0.38993978416666686

In [191]:
seq = '012345678910'
idx = 6
uc = 5
lc = 1

seq[max(0, idx - uc):max(0, idx - lc)] + seq[idx + lc + 1:idx + uc + 1]

'12348910'